In [10]:
# %load_ext autoreload
%autoreload
import os, json
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats
import pickle
from datetime import datetime 
import torch
from transformers import BertTokenizer

In [5]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [6]:
config_folder = "config/"

class obj(object):
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
               setattr(self, a, [obj(x) if isinstance(x, dict) else x for x in b])
            else:
               setattr(self, a, obj(b) if isinstance(b, dict) else b)

with open(os.path.join(config_folder, "config.json")) as f:
    config = obj(json.load(f))

In [7]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 99
INFO:root:  Batch size = 12
INFO:root:  Num steps = 80
Epoch:   0%|                                                                                                         | 0/10 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "



so far epoch accuracy:  0.10101010101010101


Epoch:  10%|█████████▋                                                                                       | 1/10 [00:34<05:14, 34.91s/it]

Epoch:  20%|███████████████████▍                                                                             | 2/10 [01:14<04:51, 36.41s/it]

...


so far epoch accuracy:  0.3434343434343434


Epoch:  90%|███████████████████████████████████████████████████████████████████████████████████████▎         | 9/10 [05:50<00:39, 39.13s/it]

Epoch: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:30<00:00, 39.09s/it]


so far epoch accuracy:  0.40404040404040403
Training complete
Best val Acc: 0.404040


INFO:root:***** Running training *****
INFO:root:  Num examples = 99
INFO:root:  Batch size = 12
INFO:root:  Num steps = 80


In [8]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds

(array([[  101,  1230,   156, ...,     0,     0,     0],
        [  101,   125,   119, ...,     0,     0,     0],
        [  101,  1230,  1249, ...,     0,     0,     0],
        ...,
        [  101,  2508, 13923, ...,   126, 17713,   102],
        [  101,  2508, 13923, ...,     0,     0,     0],
        [  101,  1967,  1185, ...,     0,     0,     0]], dtype=int64),
 array([0, 6, 0, 2, 0, 7, 5, 1, 0, 1, 0, 0, 3, 4, 0, 2, 0, 5, 0, 2, 6, 0,
        5, 2, 1, 0, 5, 6, 4, 5, 3, 2, 3, 2, 6, 2, 5, 2, 4, 5, 1, 4, 1, 1,
        2, 3, 6, 2, 0, 0, 5, 1, 3, 5, 3, 0, 0, 6, 1, 6, 0, 1, 6, 1, 4, 0,
        0, 6, 6, 3, 3, 2, 0, 1, 0, 6, 4, 3, 0, 0, 2, 2, 6, 1, 5, 6, 6, 1,
        3, 6, 2, 2, 5, 6, 4, 4, 4, 1, 7], dtype=int64),
 array([0, 6, 0, 2, 0, 0, 0, 1, 0, 2, 1, 0, 3, 2, 0, 0, 0, 2, 0, 2, 1, 2,
        2, 2, 6, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0, 0, 2, 1, 0, 2, 1,
        2, 2, 0, 2, 0, 0, 2, 6, 2, 2, 2, 0, 0, 6, 6, 0, 0, 6, 2, 3, 2, 0,
        0, 6, 0, 2, 2, 2, 0, 2, 0, 6, 4, 3, 0, 1, 2, 2, 

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
for i in range(0,50):
    if dev_label_ids[i] !=  dev_preds[i]:
        print("\n predicted ", dev_preds[i], " true ", dev_label_ids[i] , " Text: "" ". join (tokenizer.convert_ids_to_tokens(train_inputs[i])))


 predicted  2  true  6 [CLS] Text:  His Text:  S Text:  ##U Text:  ##B Text:  _ Text:  B Text:  As Text:  ##th Text:  ##ma Text:  S Text:  ##U Text:  ##B Text:  _ Text:  E Text:  was Text:  treated Text:  with Text:  standing Text:  and Text:  PR Text:  ##N Text:  al Text:  ##but Text:  ##ero Text:  ##l Text:  and Text:  i Text:  ##pra Text:  ##tri Text:  ##op Text:  ##ium Text:  and Text:  a Text:  slow Text:  pre Text:  ##dn Text:  ##ison Text:  ##e Text:  tape Text:  ##r Text:  which Text:  he Text:  l Text:  completed Text:  on Text:  [ Text:  * Text:  * Text:  211 Text:  ##5 Text:  - Text:  3 Text:  - Text:  18 Text:  * Text:  * Text:  ] Text:  and Text:  he Text:  was Text:  restart Text:  ##ed Text:  on Text:  O Text:  ##B Text:  ##J Text:  _ Text:  B Text:  Ad Text:  ##va Text:  ##ir Text:  O Text:  ##B Text:  ##J Text:  _ Text:  E Text:  - Text:  patient Text:  may Text:  require Text:  additional Text:  ne Text:  ##bs Text:  on Text:  top Text:  of Text:  his Text:  standing

In [7]:
import numpy as np
np.array(all_dev_label_ids).shape, np.array(dev_preds).shape, np.array(train_preds).shape, np.array(train_label_ids).shape

((15304,), (15304,), (61014,), (61014,))

In [8]:
%autoreload
train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(all_dev_label_ids,dev_preds )


 label: [0 8 5 ... 6 1 8]

 preds: [0 8 5 ... 6 1 8]

 label: [1 6 0 ... 8 6 3]

 preds: [1 6 0 ... 8 6 3]


In [9]:
all_experiment_results = []
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               all_dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[['<__main__.obj object at 0x00000145C2150790>',
  0.028281437373610104,
  0.10090719395104036,
  0.9828714870380854,
  0.9811421345806985,
  0.9655254255851637,
  0.9513203962222588]]

In [10]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False           34
               True           216
Dosage-Drug    False           26
               True           907
Duration-Drug  False            7
               True           129
Form-Drug      False           57
               True           897
Frequency-Drug False           59
               True          1210
Reason-Drug    False           43
               True           991
Route-Drug     False           35
               True          1110
Strength-Drug  False           57
               True          1325
no relation    False           41
               True          8160

##  Try with different configuration

### Just change model from BIOR to BERT Sequence

In [11]:
config.programsettings.MODEL_NAME = "BERT_Sequence"
config.programsettings.DEBUG_PRINT = 0

In [12]:
train_label_ids, train_preds, train_loss, all_dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 50840
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir C:\Users\pnarsina\AppData\Local\Temp\2\tmp8umad1wl
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_v

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  10%|███████▎                                                                 | 1/10 [12:02<1:48:22, 722.50s/it]


so far epoch accuracy:  0.9336709607631035


Epoch:  20%|██████████████▌                                                          | 2/10 [24:03<1:36:16, 722.08s/it]


so far epoch accuracy:  0.9750876847936539


...

Epoch:  90%|█████████████████████████████████████████████████████████████████▋       | 9/10 [1:48:11<12:01, 721.01s/it]


so far epoch accuracy:  0.986265447274396


Epoch: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [2:00:12<00:00, 721.21s/it]


so far epoch accuracy:  0.9869374241977251
Training complete
Best val Acc: 0.986937


INFO:root:***** Running training *****
INFO:root:  Num examples = 15304
INFO:root:  Batch size = 12
INFO:root:  Num steps = 12750


In [13]:

train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(all_dev_label_ids,dev_preds)


 label: [5 8 6 ... 1 2 8]

 preds: [5 8 6 ... 1 2 8]

 label: [8 0 8 ... 8 8 8]

 preds: [8 0 8 ... 8 8 8]


In [14]:
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               all_dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[['<__main__.obj object at 0x00000145C2150790>',
  0.028281437373610104,
  0.10090719395104036,
  0.9828714870380854,
  0.9811421345806985,
  0.9655254255851637,
  0.9513203962222588],
 ['<__main__.obj object at 0x00000145C2150790>',
  0.029586662868419118,
  0.09663268666182578,
  0.9826565895445114,
  0.9813933562820968,
  0.9658135876808309,
  0.9538088513676909,
  array([8, 0, 8, ..., 8, 8, 8], dtype=int64),
  array([5, 8, 6, ..., 1, 2, 8], dtype=int64)]]

In [15]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False           31
               True           219
Dosage-Drug    False           25
               True           908
Duration-Drug  False            8
               True           128
Form-Drug      False           60
               True           894
Frequency-Drug False           53
               True          1216
Reason-Drug    False           39
               True           995
Route-Drug     False           42
               True          1103
Strength-Drug  False           60
               True          1322
no relation    False           38
               True          8163

In [16]:

all_model_results_pickle_file = config.programsettings.REPORTS_DIR + "multi_model_experiment_results_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(all_model_results_pickle_file, "wb") as f:
    pickle.dump(all_experiment_results, f)  


In [26]:
from sklearn.metrics import classification_report
print(classification_report(all_dev_label_ids,dev_preds ))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1034
           1       0.97      0.96      0.97      1145
           2       0.94      0.96      0.95      1382
           3       0.94      0.96      0.95      1269
           4       0.97      0.94      0.96       136
           5       0.95      0.94      0.94       954
           6       0.98      0.97      0.97       933
           7       0.91      0.88      0.89       250
           8       1.00      1.00      1.00      8201

    accuracy                           0.98     15304
   macro avg       0.96      0.95      0.95     15304
weighted avg       0.98      0.98      0.98     15304

